In [49]:
#import dependencies
from rotten_tomatoes_scraper.rt_scraper import MovieScraper
import pandas as pd
import time
import sys
from bs4 import BeautifulSoup
import requests
from sqlalchemy import create_engine

In [2]:
#read tsv files
rating_df = pd.read_csv("title.ratings.tsv", sep='\t')
basics_df = pd.read_csv("title.basics.tsv", sep='\t')

C:\Users\ovidi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
#filter movie file for movies and year 2021
movie_df = basics_df.loc[(basics_df['titleType'] == 'movie') & (basics_df['startYear'] == '2021')]

In [10]:
#join both files
merged_data = pd.merge(movie_df, rating_df, on="tconst")
merged_data

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0104988,movie,Neues in Wittstock,Neues in Wittstock,0,2021,\N,100,Documentary,8.4,8
1,tt0199602,movie,Holy Hollywood,Holy Hollywood,0,2021,\N,82,Comedy,6.4,30
2,tt0293429,movie,Mortal Kombat,Mortal Kombat,0,2021,\N,110,"Action,Adventure,Fantasy",6.1,159967
3,tt0326716,movie,5-25-77,'77,0,2021,\N,113,"Comedy,Drama",6.3,333
4,tt0346172,movie,Zamaanat: And Justice for All,Zamaanat: And Justice for All,0,2021,\N,\N,"Drama,Family,Mystery",3.3,149
...,...,...,...,...,...,...,...,...,...,...,...
6591,tt9896876,movie,India Sweets and Spices,India Sweets and Spices,0,2021,\N,101,Comedy,5.9,339
6592,tt9898844,movie,The Hunting,The Hunting,0,2021,\N,93,Horror,3.1,162
6593,tt9900940,movie,Scrapper,The Scrapper,0,2021,\N,87,"Crime,Drama,Thriller",6.2,531
6594,tt9907782,movie,The Cursed,Eight for Silver,0,2021,\N,113,Horror,6.0,313


In [5]:
#filter new DF for Comedies
new_df = merged_data[merged_data["genres"].str.contains("Comedy")]

In [6]:
new_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
1,tt0199602,movie,Holy Hollywood,Holy Hollywood,0,2021,\N,82,Comedy,6.4,30
3,tt0326716,movie,5-25-77,'77,0,2021,\N,113,"Comedy,Drama",6.3,333
7,tt0870154,movie,Jungle Cruise,Jungle Cruise,0,2021,\N,127,"Action,Adventure,Comedy",6.6,166505
21,tt10037934,movie,One Moment,One Moment,0,2021,\N,114,Comedy,8.3,23
27,tt10042868,movie,Generation Wrecks,Generation Wrecks,0,2021,\N,112,Comedy,9.2,110
...,...,...,...,...,...,...,...,...,...,...,...
6574,tt9849004,movie,Happily,Happily,0,2021,\N,96,"Comedy,Crime,Romance",5.3,3530
6578,tt9857850,movie,A Guide to Gunfighters of the Wild West,A Guide to Gunfighters of the Wild West,0,2021,\N,74,"Comedy,Western",4.3,91
6586,tt9890120,movie,Resurrection Corporation,Resurrection Corporation,0,2021,\N,77,"Animation,Comedy,Horror",6.9,9
6590,tt9895024,movie,Heiko's World,Heikos Welt,0,2021,\N,118,"Comedy,Drama,Sport",8.6,36


In [33]:
#filter for comedy movie with rating grater than 7 and number of votes greater than 10k
top_df = new_df.loc[(new_df['averageRating'] > 7.1) & (new_df['numVotes'] > 10000)]
top_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
163,tt10366460,movie,CODA,CODA,0,2021,\N,111,"Comedy,Drama,Music",8.1,40027
171,tt10370710,movie,The Worst Person in the World,Verdens verste menneske,0,2021,\N,127,"Comedy,Drama,Romance",8.0,12169
418,tt10895576,movie,Mimi,Mimi,0,2021,\N,132,"Comedy,Drama",8.0,35951
646,tt11271038,movie,Licorice Pizza,Licorice Pizza,0,2021,\N,133,"Comedy,Drama,Romance",7.9,22117
657,tt11286314,movie,Don't Look Up,Don't Look Up,0,2021,\N,138,"Comedy,Drama,Sci-Fi",7.3,400142
728,tt11374902,movie,Doctor,Doctor,0,2021,\N,148,"Action,Comedy,Crime",7.6,13455
1510,tt12801262,movie,Luca,Luca,0,2021,\N,95,"Adventure,Animation,Comedy",7.5,134468
2113,tt13529034,movie,Meenakshi Sundareshwar,Meenakshi Sundareshwar,0,2021,\N,140,"Comedy,Romance",7.2,16628
5532,tt2953050,movie,Encanto,Encanto,0,2021,\N,102,"Adventure,Animation,Comedy",7.3,121262
5549,tt3228774,movie,Cruella,Cruella,0,2021,\N,134,"Adventure,Comedy,Crime",7.4,208348


In [38]:
#insert new column to generate the order in the table
top_df['Rank']=range(1, len(top_df)+1)
top_df

<ipython-input-38-72c79601032f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_df['Rank']=range(1, len(top_df)+1)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Rank
163,tt10366460,movie,CODA,CODA,0,2021,\N,111,"Comedy,Drama,Music",8.1,40027,1
171,tt10370710,movie,The Worst Person in the World,Verdens verste menneske,0,2021,\N,127,"Comedy,Drama,Romance",8.0,12169,2
418,tt10895576,movie,Mimi,Mimi,0,2021,\N,132,"Comedy,Drama",8.0,35951,3
646,tt11271038,movie,Licorice Pizza,Licorice Pizza,0,2021,\N,133,"Comedy,Drama,Romance",7.9,22117,4
657,tt11286314,movie,Don't Look Up,Don't Look Up,0,2021,\N,138,"Comedy,Drama,Sci-Fi",7.3,400142,5
728,tt11374902,movie,Doctor,Doctor,0,2021,\N,148,"Action,Comedy,Crime",7.6,13455,6
1510,tt12801262,movie,Luca,Luca,0,2021,\N,95,"Adventure,Animation,Comedy",7.5,134468,7
2113,tt13529034,movie,Meenakshi Sundareshwar,Meenakshi Sundareshwar,0,2021,\N,140,"Comedy,Romance",7.2,16628,8
5532,tt2953050,movie,Encanto,Encanto,0,2021,\N,102,"Adventure,Animation,Comedy",7.3,121262,9
5549,tt3228774,movie,Cruella,Cruella,0,2021,\N,134,"Adventure,Comedy,Crime",7.4,208348,10


In [34]:
#create a list with movie titles
title = top_df['primaryTitle'].tolist()
title

['CODA',
 'The Worst Person in the World',
 'Mimi',
 'Licorice Pizza',
 "Don't Look Up",
 'Doctor',
 'Luca',
 'Meenakshi Sundareshwar',
 'Encanto',
 'Cruella',
 'Ghostbusters: Afterlife',
 'Wish Dragon',
 'Free Guy',
 'The Suicide Squad',
 'Sing 2',
 'Minnal Murali',
 'The Mitchells vs the Machines',
 'The French Dispatch']

In [35]:
import urllib
ls = []
for m in title:
    movie_s = MovieScraper(movie_title= m)
    try:
        movie_s.extract_metadata()
    except urllib.error.HTTPError as exception:
        print(exception)

    ls.append(movie_s.metadata)
    time.sleep(3)

HTTP Error 404: Not Found


In [36]:
ls

[{'Score_Rotten': '96',
  'Score_Audience': '93',
  'Rating': 'PG-13',
  'Genre': ['Comedy', 'Drama']},
 {'Score_Rotten': '99',
  'Score_Audience': '86',
  'Rating': 'R',
  'Genre': ['Comedy', 'Drama', 'Romance']},
 {'Score_Rotten': '18', 'Score_Audience': '95', 'Genre': ['Comedy', 'Drama']},
 {'Score_Rotten': '91',
  'Score_Audience': '67',
  'Rating': 'R',
  'Genre': ['Comedy', 'Drama', 'Romance']},
 {},
 {'Score_Rotten': '89',
  'Score_Audience': '86',
  'Rating': 'PG-13',
  'Genre': ['Fantasy', 'Adventure', 'Action']},
 {'Score_Rotten': '91',
  'Score_Audience': '86',
  'Rating': 'PG',
  'Genre': ['Kids&family', 'Comedy', 'Fantasy', 'Animation']},
 {'Score_Rotten': '38',
  'Score_Audience': '43',
  'Genre': ['Comedy', 'Romance', 'Drama']},
 {'Score_Rotten': '90',
  'Score_Audience': '93',
  'Rating': 'PG',
  'Genre': ['Comedy', 'Kids&family', 'Animation', 'Musical', 'Fantasy']},
 {'Score_Rotten': '74',
  'Score_Audience': '97',
  'Rating': 'PG-13',
  'Genre': ['Comedy', 'Kids&famil

In [42]:
#create a dataframe based on ls and insert new column Rank to join it with initial table
rt_df = pd.DataFrame(ls)
rt_df['Rank']=range(1, len(top_df)+1)
rt_df

,Score_Rotten,Score_Audience,Rating,Genre,Rank
0,96,93,PG-13,"[Comedy, Drama]",1
1,99,86,R,"[Comedy, Drama, Romance]",2
2,18,95,NaN,"[Comedy, Drama]",3
3,91,67,R,"[Comedy, Drama, Romance]",4
4,NaN,NaN,NaN,NaN,5
5,89,86,PG-13,"[Fantasy, Adventure, Action]",6
6,91,86,PG,"[Kids&family, Comedy, Fantasy, Animation]",7
7,38,43,NaN,"[Comedy, Romance, Drama]",8
8,90,93,PG,"[Comedy, Kids&family, Animation, Musical, Fant...",9
9,74,97,PG-13,"[Comedy, Kids&family]",10


In [47]:
#rename column names
rt_df.columns = ['RT_Score', 'RT_Audience_Score', 'RT_Rating', 'RT_Genre', 'Rank']
rt_df

,RT_Score,RT_Audience_Score,RT_Rating,RT_Genre,Rank
0,96,93,PG-13,"[Comedy, Drama]",1
1,99,86,R,"[Comedy, Drama, Romance]",2
2,18,95,NaN,"[Comedy, Drama]",3
3,91,67,R,"[Comedy, Drama, Romance]",4
4,NaN,NaN,NaN,NaN,5
5,89,86,PG-13,"[Fantasy, Adventure, Action]",6
6,91,86,PG,"[Kids&family, Comedy, Fantasy, Animation]",7
7,38,43,NaN,"[Comedy, Romance, Drama]",8
8,90,93,PG,"[Comedy, Kids&family, Animation, Musical, Fant...",9
9,74,97,PG-13,"[Comedy, Kids&family]",10


In [48]:
#join IMDB dataframe with RT dataframe
final_data = pd.merge(top_df, rt_df, on="Rank")
final_data

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Rank,RT_Score,RT_Audience_Score,RT_Rating,RT_Genre
0,tt10366460,movie,CODA,CODA,0,2021,\N,111,"Comedy,Drama,Music",8.1,40027,1,96,93,PG-13,"[Comedy, Drama]"
1,tt10370710,movie,The Worst Person in the World,Verdens verste menneske,0,2021,\N,127,"Comedy,Drama,Romance",8.0,12169,2,99,86,R,"[Comedy, Drama, Romance]"
2,tt10895576,movie,Mimi,Mimi,0,2021,\N,132,"Comedy,Drama",8.0,35951,3,18,95,NaN,"[Comedy, Drama]"
3,tt11271038,movie,Licorice Pizza,Licorice Pizza,0,2021,\N,133,"Comedy,Drama,Romance",7.9,22117,4,91,67,R,"[Comedy, Drama, Romance]"
4,tt11286314,movie,Don't Look Up,Don't Look Up,0,2021,\N,138,"Comedy,Drama,Sci-Fi",7.3,400142,5,NaN,NaN,NaN,NaN
5,tt11374902,movie,Doctor,Doctor,0,2021,\N,148,"Action,Comedy,Crime",7.6,13455,6,89,86,PG-13,"[Fantasy, Adventure, Action]"
6,tt12801262,movie,Luca,Luca,0,2021,\N,95,"Adventure,Animation,Comedy",7.5,134468,7,91,86,PG,"[Kids&family, Comedy, Fantasy, Animation]"
7,tt13529034,movie,Meenakshi Sundareshwar,Meenakshi Sundareshwar,0,2021,\N,140,"Comedy,Romance",7.2,16628,8,38,43,NaN,"[Comedy, Romance, Drama]"
8,tt2953050,movie,Encanto,Encanto,0,2021,\N,102,"Adventure,Animation,Comedy",7.3,121262,9,90,93,PG,"[Comedy, Kids&family, Animation, Musical, Fant..."
9,tt3228774,movie,Cruella,Cruella,0,2021,\N,134,"Adventure,Comedy,Crime",7.4,208348,10,74,97,PG-13,"[Comedy, Kids&family]"


In [50]:
#conect to local DB
rds_connection_string = f'postgres:luca2013@:5432/imdb_RT'
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

<ipython-input-50-6b143b757d22>:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


[]

In [51]:
#Use pandas to load DataFrame into database
final_data.to_sql(name = 'movie_list', con=engine, if_exists='append', index=False)

In [52]:
#confirm data has been added
pd.read_sql_query('select * from movie_list', con=engine).head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Rank,RT_Score,RT_Audience_Score,RT_Rating,RT_Genre
0,tt10366460,movie,CODA,CODA,0,2021,\N,111,"Comedy,Drama,Music",8.1,40027,1,96,93,PG-13,"{Comedy,Drama}"
1,tt10370710,movie,The Worst Person in the World,Verdens verste menneske,0,2021,\N,127,"Comedy,Drama,Romance",8.0,12169,2,99,86,R,"{Comedy,Drama,Romance}"
2,tt10895576,movie,Mimi,Mimi,0,2021,\N,132,"Comedy,Drama",8.0,35951,3,18,95,None,"{Comedy,Drama}"
3,tt11271038,movie,Licorice Pizza,Licorice Pizza,0,2021,\N,133,"Comedy,Drama,Romance",7.9,22117,4,91,67,R,"{Comedy,Drama,Romance}"
4,tt11286314,movie,Don't Look Up,Don't Look Up,0,2021,\N,138,"Comedy,Drama,Sci-Fi",7.3,400142,5,None,None,None,None
